In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models

data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')
data.head()

Dataset shape: (404134, 18)


,time,source,destination,length,info,transmission_rate_per_1000_ms,reception_rate_per_1000_ms,transmission_average_per_sec,reception_average_per_sec,transmission_count_per_sec,reception_count_per_sec,transmission_total_duration_per_sec,reception_total_duration_per_sec,dao,dis,dio,category,label
0,0.037,39,9999,0.0,1.0,0.000000,0.671176,0.000000,0.499879,0.000000,0.671176,0.539313,0.570032,0.0,0.0,0.000000,Normal,0
1,0.037,39,9999,0.0,1.0,0.000000,0.649873,0.000000,0.505234,0.000000,0.649873,0.264704,0.530547,0.0,0.0,0.000000,Normal,0
2,0.038,39,9999,0.0,1.0,0.671176,0.652361,0.462516,0.501327,0.671768,0.652361,0.546376,1.000000,0.0,0.0,0.690115,Blackhole,1
3,0.045,39,9999,0.0,1.0,0.000000,0.633786,0.000000,0.517346,0.000000,0.634105,0.585425,0.553276,0.0,0.0,0.000000,Normal,0
4,0.046,39,9999,0.0,1.0,0.000000,0.630378,0.000000,0.538789,0.000000,0.630378,0.443171,0.615377,0.0,0.0,0.000000,Normal,0


In [2]:
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample minority class (blackhole)
upsampled = resample(blackhole, 
                               replace=True,    # sample with replacement
                               n_samples=len(normal), # match number in majority class
                               random_state=42) # reproducible results

# Combine majority and upsampled minority
# Identify and inspect categorical columns
print(upsampled.dtypes)

# If there are any categorical columns, convert them to numerical using one-hot encoding
upsampled_encoded = pd.get_dummies(upsampled, drop_first=True)

# Preprocessing: feature scaling after encoding
X = upsampled_encoded.drop('label', axis=1)
y = upsampled_encoded['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


time                                   float64
source                                   int64
destination                              int64
length                                 float64
info                                   float64
transmission_rate_per_1000_ms          float64
reception_rate_per_1000_ms             float64
transmission_average_per_sec           float64
reception_average_per_sec              float64
transmission_count_per_sec             float64
reception_count_per_sec                float64
transmission_total_duration_per_sec    float64
reception_total_duration_per_sec       float64
dao                                    float64
dis                                    float64
dio                                    float64
category                                object
label                                    int64
dtype: object


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Bayesian Neural Network Model using Monte Carlo Dropout
class BayesianNN(tf.keras.Model):
    def __init__(self):
        super(BayesianNN, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.dropout1 = layers.Dropout(0.5)  # Dropout layer for uncertainty
        self.fc2 = layers.Dense(32, activation='relu')
        self.dropout2 = layers.Dropout(0.5)  # Dropout layer for uncertainty
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.fc1(inputs)
        x = self.dropout1(x, training=training)  # Apply dropout during training
        x = self.fc2(x)
        x = self.dropout2(x, training=training)  # Apply dropout during training
        return self.out(x)

# Model training setup for Bayesian Neural Network
def bayesian_train_step(model, optimizer, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)  # Enable training mode for dropout
        predictions = tf.squeeze(predictions)  # Ensure predictions have the same shape as y
        loss = tf.keras.losses.binary_crossentropy(y, predictions)
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Learning setup remains unchanged
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = BayesianNN()  # Use BayesianNN model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            bayesian_train_step(global_model, optimizer, X, y)

    return global_model

# Simulate distributed datasets for federated learning
X_splits = np.array_split(X_train, 5)
y_splits = np.array_split(y_train, 5)

# Federated training
global_model = federated_train(X_splits, y_splits)


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10


In [4]:
y_pred = global_model(X_test)
y_pred = np.round(y_pred)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Confusion Matrix:
[[53971]]

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     53971

    accuracy                           1.00     53971
   macro avg       1.00      1.00      1.00     53971
weighted avg       1.00      1.00      1.00     53971


Accuracy Score:
1.0


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
